# Instrument classifier

## Imports

In [16]:
import scikitplot as skplt
import soundfile as sf
import numpy as np
#import librosa
#import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import math
import antropy as ant
import seaborn as sns

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (r2_score,
                             roc_auc_score,
                             classification_report,
                             confusion_matrix,
                             roc_curve,
                             accuracy_score,
                             mean_squared_error)
from sklearn.tree import (DecisionTreeClassifier,
                          plot_tree)
from sklearn.ensemble import RandomForestClassifier

from scipy import signal
from scipy.fft import fft, fftfreq, rfftfreq, rfft
from scipy.stats import norm, kurtosis

from pedalboard import LadderFilter, Pedalboard
from soundfile import SoundFile

## Functions

In [17]:
# Centroid
def centroid(frequency, amplitude):
    centroid = np.sum(amplitude * frequency) / np.sum(amplitude)
    return centroid

# Spread
def spread(frequency, amplitude):
    centroid = np.sum(amplitude * frequency) / np.sum(amplitude)
    spread = math.sqrt(np.sum((frequency - centroid) ** 2 * amplitude) / np.sum(amplitude))
    return spread

# Skewness
def skewness(frequency, amplitude):
    centroid = np.sum(amplitude * frequency) / np.sum(amplitude)
    spread = math.sqrt(np.sum((frequency - centroid) ** 2 * amplitude) / np.sum(amplitude))
    skewness = np.sum((frequency - centroid) ** 3 * amplitude) / (spread ** 3 * np.sum(amplitude))
    return skewness

# Kurtosis
def kurtosis(frequency, amplitude):
    centroid = np.sum(amplitude * frequency) / np.sum(amplitude)
    spread = math.sqrt(np.sum((frequency - centroid) ** 2 * amplitude) / np.sum(amplitude))
    kurtosis = np.sum((frequency - centroid) ** 4 * amplitude) / (spread ** 4 * np.sum(amplitude)) - 3
    return kurtosis

# Entropy
def entropy(amplitude):
    entropy = -np.sum(amplitude * np.log(amplitude)) / np.log(len(amplitude))
    return entropy

# Flatness
def flatness(amplitude):
    flatness = np.exp(np.mean(np.log(amplitude))) / np.mean(amplitude)
    return flatness

# Decrease
def decrease(amplitude):
    first = amplitude[0]
    amplitude = np.delete(amplitude, 0)
    decrease = np.sum((amplitude - first) / (len(amplitude) - 1)) / np.sum(amplitude)
    return decrease

# Slope
def slope(frequency, amplitude):
    slope = sum((frequency - np.mean(frequency)) * (amplitude - np.mean(amplitude))) / np.sum((frequency - np.mean(frequency)) ** 2)
    return slope

# Peak
def peak(frequency, amplitude):
    index = np.argmax(np.array(amplitude))
    peak = frequency[index]
    return peak

# Crest
def crest(frequency, amplitude):
    index = np.argmax(np.array(amplitude))
    peak = frequency[index]
    crest = peak / np.mean(amplitude)
    return crest

# Rolloff
def rolloff(amplitude, treshold):
    if treshold is None:
        treshold = 0.95
    index = np.where(x > amplitude[end] * 0.95, x, np.cumsum(amplitude))
    rolloff = amplitude[index[0]]
    return rolloff

## Instruments

In [21]:
instruments = ["Kick", "Snare", "Hihat", "Guitar", "Bass", "Vox"]
numSounds = 50

## Audio load

In [26]:
generalPath = "../InstrumentsClassifier/Instruments/"

instrumentsAudio = []
for instrument in instruments:
    instrumentPath = generalPath + instrument + "/"
    sound = []
    
    for i in range(numSounds):
        path = instrumentPath + instrument + "_" + str(i+1) + ".wav"
        audio = SoundFile(path)

        if audio.channels == 1:
            samples, sr = sf.read(path)
            sound.append([samples, sr])
        else:
            samples, sr = sf.read(path)
            sound.append([samples[:,0], sr])
            
    instrumentsAudio.append([instrument, sound])

## Pre-processing

### Kicks

In [25]:
kick = []

count = 0
for kick in kicks:
    amplitude = kick[0]
    sr = kick[1]
    time = np.arange(0, kick[0][end], sr)
    
    spectrum = np.abs(rfft(amplitude))
    frequency = rfftfreq(len(amplitude), 1 / sr)
    
    print("Amplitude:", amplitude)
    print("Time:", time)
    print("Spectrum:", spectrum)
    print("Frequency:", frequency)

NameError: name 'kicks' is not defined

In [32]:
sounds_info = []

cont = 0
for instrument in instruments:
    sound = []
    
    for i in range(numSounds):    
        fila = []
        
        amplitude = instrumentsAudio[cont][1][i][0]
        sr = instrumentsAudio[cont][1][i][1]
        time = np.arange(0, len(amplitude), 1) / sr
    
        spectrum = np.abs(rfft(amplitude))
        frequency = rfftfreq(len(amplitude), 1 / sr)
        
        if i == 0:
            print("Amplitude:", amplitude)
            print("Time:", time)
            print("Spectrum:", spectrum)
            print("Frequency:", frequency)
        
    cont += 1

Amplitude: [-1.00000000e+00 -7.56968975e-01 -6.05895996e-01 ... -4.76837158e-06
 -5.48362732e-06  2.86102295e-06]
Time: [0.00000000e+00 2.26757370e-05 4.53514739e-05 ... 4.83809524e-01
 4.83832200e-01 4.83854875e-01]
Spectrum: [12.88220441 10.87964072 17.42807898 ...  1.29862327  0.58276304
  0.21440064]
Frequency: [0.00000000e+00 2.06663855e+00 4.13327710e+00 ... 2.20448334e+04
 2.20469000e+04 2.20489667e+04]
Amplitude: [-1.00000000e+00 -1.00000000e+00 -1.55202270e-01 ... -3.21865082e-06
  6.07967377e-06 -2.62260437e-06]
Time: [0.00000000e+00 2.26757370e-05 4.53514739e-05 ... 4.83809524e-01
 4.83832200e-01 4.83854875e-01]
Spectrum: [2.00111055 1.85357798 1.77480598 ... 0.60853508 0.60733269 0.60748508]
Frequency: [0.00000000e+00 2.06663855e+00 4.13327710e+00 ... 2.20448334e+04
 2.20469000e+04 2.20489667e+04]
Amplitude: [-3.57627869e-07 -4.60839272e-03  1.51405573e-01 ... -1.13248825e-05
 -5.48362732e-06 -6.31809235e-06]
Time: [0.00000000e+00 2.26757370e-05 4.53514739e-05 ... 2.3537415